In [6]:
import gdal
import numpy as np

#### create your own population count manually

In [7]:
# read in population count
path = '/nfs/a68/earlacoa/population/count/'
population_tif = gdal.Open(
    path + 'gpw-v4-population-count-adjusted-to-2015-unwpp-country-totals-2015-qtr-deg.tif'
)
population_count = population_tif.ReadAsArray()

# transform lat and lon
geo_transform = population_tif.GetGeoTransform()
population_x = np.linspace(
    geo_transform[0], 
    geo_transform[0] + geo_transform[1] * population_count.shape[1], 
    population_count.shape[1]
)
population_y = np.linspace(
    geo_transform[3], 
    geo_transform[3] + geo_transform[5] * population_count.shape[0], 
    population_count.shape[0]
)
population_xx, population_yy = np.meshgrid(population_x, population_y)

# replace negative fill values
population_count[population_count < 0] = 0.0

# older verions (< v4) count/lat is upside down
#population_count = np.flipud(population_count)

# save
np.savez_compressed(
    path + 'gpw-v4-population-count-adjusted-to-2015-unwpp-country-totals-2015-qtr-deg.npz', 
    population_count=population_count,
    population_xx=population_xx,
    population_yy=population_yy
)

#### check this manual version is the same as the imported .npz version

In [16]:
def import_npz(npz_file, namespace):
    '''load all numpy arrays into global namespace'''
    '''ensure original arrays have the variable name you require'''
    data = np.load(npz_file)
    for var in data:
        if data[var].dtype == np.dtype('float64'):
            namespace[var] = data[var].astype('float32')
        else:
            namespace[var] = data[var]

In [17]:
# import pop_z_2015
import_npz('/nfs/a68/earlacoa/population/count/population-count-0.25deg.npz', globals())

In [34]:
print((pop_z_2015 == population_array).all())

True
